# Massive AAPL Daily Bars Processor
- Loads raw table from Copy activity
- Adds Date from timestamp
- Renames columns to clean names
- Dedups on Date
- Overwrites table

In [ ]:
from pyspark.sql.functions import col, to_date

# Load the existing final table (if it exists)
try:
    existing_df = spark.read.table("aapl_daily_massive")
except:
    existing_df = spark.createDataFrame([], schema="ticker STRING, volume LONG, vwap DOUBLE, open_price DOUBLE, close_price DOUBLE, high_price DOUBLE, low_price DOUBLE, transactions LONG, Date DATE")  # empty if table doesn't exist yet

# Load the new batch from Notebook 1 (flattened/cleaned)
new_df = spark.read.table("DailyStockData_Clean")

# Add Date if missing (from timestamp)
new_df = new_df.withColumn("Date", to_date(col("timestamp"))) \
               .drop("timestamp")

# Rename columns in new batch (to match existing table)
new_df = new_df.withColumnRenamed("v", "volume") \
               .withColumnRenamed("vw", "vwap") \
               .withColumnRenamed("o", "open_price") \
               .withColumnRenamed("c", "close_price") \
               .withColumnRenamed("h", "high_price") \
               .withColumnRenamed("l", "low_price") \
               .withColumnRenamed("n", "transactions")

# Combine existing + new data
combined_df = existing_df.unionByName(new_df, allowMissingColumns=True)

# Dedup on Date (keep latest version if duplicates)
combined_df = combined_df.dropDuplicates(["Date"])

# Show preview of latest data
combined_df.orderBy("Date", ascending=False).show(10, truncate=False)

# Overwrite the final table with combined, deduped data
combined_df.write.mode("overwrite") \
  .option("overwriteSchema", "true") \
  .saveAsTable("aapl_daily_massive")

print("Daily cleanup complete: new data appended, deduped on Date, table updated. Row count:", combined_df.count())

StatementMeta(, , -1, SessionError, , SessionError)

InvalidHttpRequestToLivy: [TooManyRequestsForCapacity] This spark job can't be run because you have hit a spark compute or API rate limit. To run this spark job, cancel an active Spark job through the Monitoring hub, choose a larger capacity SKU, or try again later. HTTP status code: 430 {Learn more} HTTP status code: 430.

Last tested: 2026-01-21
Columns: Ticker, Open, High, Low, Close, Volume, Date